In [1]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm
from gensim.models import KeyedVectors
import gc

In [2]:
train_path = '../../user_data/model_1/'
test_path = '../../user_data/model_1/'
header = 'model_1'
now_phase = 9

In [3]:
file_name = 'recall_0531_addsim_addAA_RA_additemtime_addcount_addnn_addtxt'
df = pd.read_csv(train_path + 'new_recall/' + file_name + '.csv')

In [4]:
click_trn = []
click_tst = []
#qtime_tst = []
for p in tqdm(range(0, now_phase+1)):
    tmp = pd.read_csv(train_path + header + f'_train_click-{p}.csv', header=None, names=['user_id', 'item_id', 'time'])
    tmp['phrase'] = p
    click_trn.append(tmp)
    tmp = pd.read_csv(test_path + header + f'_test_click-{p}.csv', header=None, names=['user_id', 'item_id', 'time'])
    tmp['phrase'] = p
    click_tst.append(tmp)
    #tmp = pd.read_csv(test_path + header + f'_test_qtime-{p}.csv', header=None, names=['user_id', 'item_id', 'query_time'])
    #tmp['phrase'] = p
    #qtime_tst.append(tmp)
    
click_trn = pd.concat(click_trn, axis=0, ignore_index=True)
click_tst = pd.concat(click_tst, axis=0, ignore_index=True)
#qtime_tst = pd.concat(qtime_tst, axis=0, ignore_index=True)

100%|██████████| 10/10 [00:00<00:00, 11.50it/s]


In [5]:
click_df = pd.concat([click_trn, click_tst], axis=0, ignore_index=True)
click_df['item_count'] = click_df.groupby(['item_id', 'phrase'])['user_id'].transform('count')

In [6]:
click_df.shape

(2811547, 5)

In [7]:
click_df.head()

user_id  item_id      time  phrase  item_count
0     4965       18  0.983763       0           4
1    20192       34  0.983772       0          12
2    30128       91  0.983780       0           9
3    29473      189  0.983930       0           4
4    10625      225  0.983925       0           5

In [8]:
count_map = click_df[['item_id', 'phrase', 'item_count']].drop_duplicates()

In [9]:
df.columns

Index(['user_id', 'item_id', 'sim', 'feature_0', 'feature_1', 'feature_2',
       'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7',
       'feature_8', 'feature_9', 'feature_10', 'cn_sim', 'hpi_sim', 'hdi_sim',
       'lhn1_sim', 'ra_sim', 'aa_sim', 'time_interval', 'item_count_12h',
       'item_count_6h', 'item_count_4h', 'item_count_2h', 'item_count_1h',
       'phrase', 'count', 'diff_from_last', 'diff_from_next', 'diff_from_max',
       'nn', 'txt_feature_0', 'txt_feature_1', 'txt_feature_2',
       'txt_feature_3', 'txt_feature_4', 'img_feature_0', 'img_feature_1',
       'img_feature_2', 'img_feature_3', 'img_feature_4'],
      dtype='object')

In [10]:
def gen_add_1(df):
    group_df = df.groupby(['user_id', 'phrase', 'item_count'])['time'].agg([['user_item_count_cnt', 'count'],
                                                                            ['user_item_count_max_time', 'max'],
                                                                            ['user_item_count_min_time', 'min']]).reset_index()
    group_df['sum'] = group_df.groupby(['user_id', 'phrase'])['user_item_count_cnt'].transform('sum')
    group_df['user_item_count_ratio'] = group_df['user_item_count_cnt'] / group_df['sum']
    group_df['user_item_count_timedelta'] = group_df['user_item_count_max_time'] - group_df['user_item_count_min_time']
    del group_df['sum']
    return group_df

In [11]:
df = df.merge(count_map, on=['item_id', 'phrase'], how='left')

In [12]:
df.shape

(15397325, 43)

In [13]:
train_add1 = gen_add_1(click_df)

In [14]:
train_add1.shape

(1554274, 8)

In [15]:
df = df.merge(train_add1, on=['user_id', 'phrase', 'item_count'], how='left')

In [16]:
df.shape

(15397325, 48)

user_id  item_id     sim  feature_0     feature_1  feature_2  \
0              11    22042  0.0009       40.0  2.778323e+14     0.0332   
1              11    55774  0.0009        2.0  9.690100e+00     0.1340   
2              11     3121  0.0008       12.0  7.111409e+03     0.4982   
3              11   113564  0.0006        1.0  3.872000e-01    12.2916   
4              11    20389  0.0005       26.0  7.516398e+07     0.0147   
...           ...      ...     ...        ...           ...        ...   
15397320    35429    18993  0.0000        1.0  1.723000e-01     0.0631   
15397321    35429    65273  0.0000        1.0  2.879000e-01     0.1349   
15397322    35429   109983  0.0000        1.0  2.495000e-01     0.0008   
15397323    35429    77342  0.0000        1.0  1.679000e-01     0.0262   
15397324    35429     8699  0.0000        1.0  2.569000e-01     0.0492   

          feature_3  feature_4  feature_5  feature_6  ...  img_feature_1  \
0               1.0     0.1570   487.0744          2  ...       0.198536   
1               1.0     0.0000   490.7688          0  ...       0.268509   
2               1.0     0.1570   361.1106          2  ...       0.267309   
3               1.0     0.0000    82.4804          0  ...       0.166400   
4               1.0     0.0287   507.9350          1  ...       0.232282   
...             ...        ...        ...        ...  ...            ...   
15397320        3.0     1.0325   346.9613          2  ...       0.097220   
15397321        1.0     1.0325    63.3729          2  ...       0.454160   
15397322        1.0     1.0325   268.4664          2  ...       0.337540   
15397323        2.0     1.0325    61.0978          2  ...       0.204240   
15397324        1.0     1.0325   320.9214          2  ...       0.313380   

          img_feature_2  img_feature_3  img_feature_4  item_count  \
0              0.100041         2.1839              4        17.0   
1              0.130411         2.9536              4         7.0   
2              0.172974         2.9404              4         NaN   
3              0.143937         1.8304              4         4.0   
4              0.199696         2.5551              4         6.0   
...                 ...            ...            ...         ...   
15397320       0.054046         0.4861              0         NaN   
15397321       0.170445         2.2708              0         NaN   
15397322       0.096794         1.6877              0         4.0   
15397323       0.066843         1.0212              0        11.0   
15397324       0.106111         1.5669              0         NaN   

          user_item_count_cnt  user_item_count_max_time  \
0                         NaN                       NaN   
1                         1.0                  0.983868   
2                         NaN                       NaN   
3                         2.0                  0.983868   
4                         NaN                       NaN   
...                       ...                       ...   
15397320                  NaN                       NaN   
15397321                  NaN                       NaN   
15397322                  1.0                  0.984311   
15397323                  NaN                       NaN   
15397324                  NaN                       NaN   

          user_item_count_min_time  user_item_count_ratio  \
0                              NaN                    NaN   
1                         0.983868                   0.10   
2                              NaN                    NaN   
3                         0.983830                   0.20   
4                              NaN                    NaN   
...                            ...                    ...   
15397320                       NaN                    NaN   
15397321                       NaN                    NaN   
15397322                  0.984311                   0.25   
15397323                       NaN                    NaN   

In [17]:
df.to_csv(train_path + 'new_recall/' + file_name + '_interactive.csv', index=False)